In [5]:
import sys
sys.path.append(r"c:\BAO\metaheuristics")

from nsga.main import NSGAII
import networkx as nx
import pandas as pd
import random

In [14]:
import sys
sys.path.append(r"c:\BAO\metaheuristics")

import os
from nsga.main import NSGAII
import osmnx as ox
import pandas as pd
import networkx as nx
import random

# 1. Cargar el grafo y los datos de las aristas
grafo_path = r"c:\BAO\data\madrid.graphml"
edges_path = r"c:\BAO\data\edges_clean.json"

# Verificar que los archivos existen
assert os.path.exists(grafo_path), f"El archivo {grafo_path} no existe."
assert os.path.exists(edges_path), f"El archivo {edges_path} no existe."

print("Cargando grafo desde archivo...")
graph = ox.load_graphml(grafo_path)

print("Cargando datos de las aristas...")
edges_gdf = pd.read_json(edges_path)

# 2. Forzar la ejecución con la ruta 1
# Datos de la ruta 1 (reemplaza estos valores con los correctos si los tienes)
source = 25552320  # osmid del nodo de origen de la ruta 1
target = 996184096  # osmid del nodo de destino de la ruta 1

print(f"Forzando la ejecución con la ruta 1: Nodo origen {source}, Nodo destino {target}")

# Verificar si existe una ruta más corta entre los nodos
try:
    shortest_path = nx.shortest_path(graph, source=source, target=target, weight="length")
    print(f"Ruta más corta encontrada: {shortest_path}")
except nx.NetworkXNoPath:
    print("No existe una ruta entre el origen y el destino.")
    shortest_path = []

# 3. Configurar y ejecutar el algoritmo NSGA-II
if shortest_path:
    print("Inicializando NSGA-II...")
    population_size = 1
    generations = 5
    route_length = len(shortest_path)  # Usar la longitud de la ruta más corta como límite

    nsga2 = NSGAII(
        graph=graph,
        edges_gdf=edges_gdf,
        source=source,
        target=target,
        population_size=population_size,
        generations=generations,
        route_length=route_length
    )

    print("Ejecutando NSGA-II...")
    nsga2.evolve()

    # 4. Mostrar las soluciones finales
    print("\nSoluciones finales:")
    for route in nsga2.population:
        objectives = nsga2.evaluate_objectives(route)
        print(f"Ruta: {route}")
        print(f"Objetivos (Distancia, Tiempo, Consumo): {objectives}")

    # 5. Visualizar las rutas
    import matplotlib.pyplot as plt

    print("Visualizando el grafo y las rutas...")
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
    nx.draw_networkx_edge_labels(graph, pos, edge_labels={(u, v): f"{d['length']}m" for u, v, d in graph.edges(data=True)})

    # Dibujar las rutas del frente de Pareto
    for route in nsga2.population:
        best_nodes = [u for u, v, k in route] + [route[-1][1]]
        nx.draw_networkx_nodes(graph, pos, nodelist=best_nodes, node_color='orange', node_size=700)
    plt.show()
else:
    print("No se puede ejecutar NSGA-II porque no existe una ruta válida entre los nodos seleccionados.")

Cargando grafo desde archivo...
Cargando datos de las aristas...
Forzando la ejecución con la ruta 1: Nodo origen 25552320, Nodo destino 996184096
Ruta más corta encontrada: [25552320, 25552322, 25552324, 150541906, 25552326, 25552369, 25552357, 25552336, 25552371, 25552373, 150540591, 996184096]
Inicializando NSGA-II...
Ejecutando NSGA-II...


ValueError: Sample larger than population or is negative